In [25]:
def crear_matriz_secuencias(secuencia):
    matriz = []

    # Iterar sobre cada secuencia
    for secuencia in secuencia:
        fila = []
        # Iterar sobre cada letra de la secuencia
        for letra in secuencia:
            fila.append(letra)
        matriz.append(fila)

    return matriz

# Definir Secuencias
Secuencias= ["Murciélago", "Solidaridad", "capacitación"]
matriz_Inicial = crear_matriz_secuencias(Secuencias)

def cuadruplicar_matriz(matriz):
    # Duplicar la matriz original dos veces
    matriz_duplicada = [fila[:] for fila in matriz]
    matriz_duplicada1 = [fila[:] for fila in matriz]
    matriz_duplicada2 = [fila[:] for fila in matriz]
    matriz_duplicada3 = [fila[:] for fila in matriz]

    return matriz_duplicada, matriz_duplicada1, matriz_duplicada2,matriz_duplicada3

matriz_1, matriz_2, matriz_3, matriz_4,= cuadruplicar_matriz(matriz_Inicial)
# Crear la población inicial
poblacion_inicial = [matriz_Inicial, matriz_1, matriz_2, matriz_3,matriz_4]
# Imprimir las matrices en la población inicial
print("Poblacion Inicial\n")
for i, matriz in enumerate(poblacion_inicial):
    print(f"Matriz {i + 1}:")
    for fila in matriz:
        print(fila)
    print()

import random

# Mutar Matriz agregar 2 gaps '-' en cada fila aleatoriamenete
def mutar_matriz(matriz):
    matriz_con_guiones = []

    for fila in matriz:
        nueva_fila = fila[:]  # Creamos una copia de la fila original
        # Agregamos dos guiones en diferentes posiciones
        for _ in range(2):
            posicion = random.randint(0, len(nueva_fila) - 1)
            nueva_fila.insert(posicion, "-")
        matriz_con_guiones.append(nueva_fila)

    return matriz_con_guiones

poblacion_mutada = []

#Funcion para Evaluar la poblacion indiviualmente
def evaluar_matriz(matriz):
    puntuacion_total = 0
    num_filas = len(matriz)
    num_columnas = len(matriz[0])

    for col in range(num_columnas):
        letras_vistas = set()  
        for fila in range(num_filas):
            letra_actual = matriz[fila][col]
            if letra_actual != "-" and letra_actual in letras_vistas:
                puntuacion_total += 20
            letras_vistas.add(letra_actual)

    return puntuacion_total

#Funcion para Calular promedio de la Poblacion Total
def promedio_puntuaciones(poblacion):
    if not poblacion:  # Si la población está vacía, retorna una lista vacía
        return []

    total_puntuaciones = sum(evaluar_matriz(matriz) for matriz in poblacion)
    promedio = total_puntuaciones / len(poblacion)
    # Conservar solo las matrices cuya puntuación sea mayor o igual al promedio
    poblacion_final = [matriz for matriz in poblacion if evaluar_matriz(matriz) >= promedio]
    # Si no hay matrices disponibles para formar parejas, mantener solo las matrices 
    # cuya puntuación sea mayor o igual al promedio
    if len(poblacion_final) <= 1:
        return poblacion_final
    # Calcular el número de matrices conservadas
    num_matrices_conservadas = len(poblacion_final)
    # Si el número de matrices conservadas no es divisible entre 2, seleccionar una matriz 
    # aleatoriamente de las no conservadas
    if num_matrices_conservadas % 2 != 0:
        matrices_no_conservadas = [matriz for matriz in poblacion if matriz 
                                   not in poblacion_final]
        matriz_seleccionada = random.choice(matrices_no_conservadas)
        poblacion_final.append(matriz_seleccionada)

    return poblacion_final


def contar_caracteres(matriz):
    conteo_caracteres = []
    for fila in matriz:
        conteo = sum(1 for char in fila if char != '-')
        partes = conteo // 3
        conteo_caracteres.append(partes)
    return conteo_caracteres

#Funcion para dividir las marices en tres parte 
def dividir_filas(matrices):
    matrices_divididas = []
    for matriz in matrices:
        filas_divididas = []
        cantidades_caracteres = contar_caracteres(matriz)
        
        for fila, cantidad in zip(matriz, cantidades_caracteres):
            contador = 0
            posiciones = []
            for i, caracter in enumerate(fila):
                if caracter != "-":
                    contador += 1
                    if contador == cantidad:
                        posiciones.append(i)
                        contador = 0
            
            if len(posiciones) < 2:
                posiciones.extend([len(fila)-1, len(fila)-1])
            
            partes = [fila[:posiciones[0]+1], fila[posiciones[0]+1:posiciones[1]+1], fila[posiciones[1]+1:]]
            filas_divididas.append(partes)
        
        matrices_divididas.append(filas_divididas)
    
    return matrices_divididas

def hacer_parejas(matrices):
    # Si no hay suficientes matrices para formar al menos una pareja, retornar una lista vacía
    if len(matrices) < 2:
        return []
    
    random.shuffle(matrices)  # Mezclamos las matrices aleatoriamente
    parejas = []
    for i in range(0, len(matrices), 2):
        if i + 1 < len(matrices):  # Si hay al menos otra matriz disponible para formar una pareja
            padre = matrices[i]
            madre = matrices[i+1]
            parejas.append((padre, madre))
    
    # Dividir las filas de las matrices en cada pareja
    parejas_divididas = []
    for pareja in parejas:
        parejas_divididas.append((dividir_filas([pareja[0]])[0], dividir_filas([pareja[1]])[0]))
    
    return parejas_divididas

#Funcion para crear hijo de cada pareja 
def crear_hijo(padre, madre):
    hijo = []
    for partes_padre, partes_madre in zip(padre, madre):
        parte1_padre, parte2_madre, parte3_padre = partes_padre
        hijo.append(parte1_padre + parte2_madre + parte3_padre)
    return hijo

# Preguntar al usuario cuántas generaciones desea tener
num_generaciones = int(input("¿Cuántas generaciones deseas tener? "))

# Iterar según el número de generaciones deseado
for _ in range(num_generaciones):
    if _ == 0:
        poblacion = poblacion_inicial
    else:
        poblacion = generacion

    poblacion_mutada = []
    for matriz in poblacion:
        matriz_mutada = mutar_matriz(matriz)
        poblacion_mutada.append(matriz_mutada)

    # Evaluar las matrices mutadas e imprimir sus puntuaciones
    print("Puntuaciones de las matrices mutadas:")
    for i, matriz in enumerate(poblacion_mutada):
        print(f"Matriz {i + 1}:")
        for fila in matriz:
            print(fila)
        puntuacion = evaluar_matriz(matriz)
        print("Puntuación:", puntuacion)
        print()

    # Verificar si la población mutada está vacía antes de calcular el promedio de puntuaciones
    if poblacion_mutada:
        # Calcular el promedio de las puntuaciones de la población mutada
        poblacionConservada = promedio_puntuaciones(poblacion_mutada)

        # Imprimir las matrices conservadas junto con sus puntuaciones
        print("\nMatrices conservadas:")
        for i, matriz in enumerate(poblacionConservada):
            print(f"Matriz {i + 1}:")
            for fila in matriz:
                print(fila)
            puntuacion = evaluar_matriz(matriz)
            print("Puntuación:", puntuacion)
            print()

        # Hacer parejas de padres y madres
        parejas = hacer_parejas(poblacionConservada)

        # Crear hijos a partir de las parejas de padres y madres
        hijos = []
        for padre, madre in parejas:
            hijo = crear_hijo(padre, madre)
            hijos.append(hijo)

        # Imprimir los hijos
        print("\nHijos:")
        for idx, hijo in enumerate(hijos, 1):
            print(f"Hijo {idx}:")
            for fila in hijo:
                print(fila)
            print()

        def unir_sublistas(matriz):
            matriz_unida = [[letra for sublista in fila for letra in sublista] for fila in matriz]
            return matriz_unida

        generacion = []
        for pareja, hijo in zip(parejas, hijos):
            generacion.append(unir_sublistas(pareja[0]))
            generacion.append(unir_sublistas(pareja[1]))
            generacion.append(unir_sublistas(hijo))

        # Imprimir la nueva generación
        print("\nNueva Generación:")
        for matriz in generacion:
            print(matriz)
    else:
        print("La población mutada está vacía en esta iteración. No se realizarán más operaciones.")



Poblacion Inicial

Matriz 1:
['M', 'u', 'r', 'c', 'i', 'é', 'l', 'a', 'g', 'o']
['S', 'o', 'l', 'i', 'd', 'a', 'r', 'i', 'd', 'a', 'd']
['c', 'a', 'p', 'a', 'c', 'i', 't', 'a', 'c', 'i', 'ó', 'n']

Matriz 2:
['M', 'u', 'r', 'c', 'i', 'é', 'l', 'a', 'g', 'o']
['S', 'o', 'l', 'i', 'd', 'a', 'r', 'i', 'd', 'a', 'd']
['c', 'a', 'p', 'a', 'c', 'i', 't', 'a', 'c', 'i', 'ó', 'n']

Matriz 3:
['M', 'u', 'r', 'c', 'i', 'é', 'l', 'a', 'g', 'o']
['S', 'o', 'l', 'i', 'd', 'a', 'r', 'i', 'd', 'a', 'd']
['c', 'a', 'p', 'a', 'c', 'i', 't', 'a', 'c', 'i', 'ó', 'n']

Matriz 4:
['M', 'u', 'r', 'c', 'i', 'é', 'l', 'a', 'g', 'o']
['S', 'o', 'l', 'i', 'd', 'a', 'r', 'i', 'd', 'a', 'd']
['c', 'a', 'p', 'a', 'c', 'i', 't', 'a', 'c', 'i', 'ó', 'n']

Matriz 5:
['M', 'u', 'r', 'c', 'i', 'é', 'l', 'a', 'g', 'o']
['S', 'o', 'l', 'i', 'd', 'a', 'r', 'i', 'd', 'a', 'd']
['c', 'a', 'p', 'a', 'c', 'i', 't', 'a', 'c', 'i', 'ó', 'n']

Puntuaciones de las matrices mutadas:
Matriz 1:
['M', 'u', 'r', 'c', 'i', 'é', 'l', 'a